# Try XGBoost
- We will skip the EDA as it is already explored in notebooks `1_SARIMA` and `2_custom_predictor`, and focus on Hyperparameter Tuning and Feature Engineering
- Reference Notebook: https://www.kaggle.com/aastha124/xgboost-hyperparameter-tuning

### Requirements:
scikit-learn==0.23.2 to avoid deprecated param `iid` error in scikit-optimize==0.81
- IMPORTANT: Remember to restart kernel after reinstalling sklearn

In [1]:
# # RUN THIS CELL ONLY IF YOU ARE ON KAGGLE KERNEL; ELSE ENSURE scikit-learn==0.23.2 IN YOUR VIRTUAL ENVIRONMENT VIA TERMINAL (NOT VIA JUPYTER NOTEBOOK)
# !pip uninstall scikit-learn -y
# !pip install scikit-learn==0.23.2

Found existing installation: scikit-learn 0.24.1
Uninstalling scikit-learn-0.24.1:
  Successfully uninstalled scikit-learn-0.24.1
     |████████████████████████████████| 6.8 MB 2.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.


In [2]:
import sklearn
print(sklearn.__version__)

0.23.2


In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from pathlib import Path

root_path = '../input/demand-forecasting-kernels-only/'
train_original = pd.read_csv(Path(root_path, 'train.csv'), low_memory=False, 
                    parse_dates=['date'], index_col=['date'])
test_original = pd.read_csv(Path(root_path, 'test.csv'), low_memory=False, 
                   parse_dates=['date'], index_col=['date'])
sample_sub_original = pd.read_csv(Path(root_path, 'sample_submission.csv'))

In [4]:
train = train_original.copy()
test = test_original.copy()

train.head()

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10


Since our `train` and `test` dataframes have the `date` column as the index, we can use the following code to easily get the new columns, `reset_index` can help restore the index later on.

In [5]:
train['weekday'] = train.index.dayofweek
train['dayofyear'] = train.index.dayofyear
train['year'] = train.index.year
train['month'] = train.index.month
train = train.reset_index()

test['weekday'] = test.index.dayofweek
test['dayofyear'] = test.index.dayofyear
test['year'] = test.index.year
test['month'] = test.index.month
test = test.reset_index()

In [6]:
train.isnull().sum()

date         0
store        0
item         0
sales        0
weekday      0
dayofyear    0
year         0
month        0
dtype: int64

Now, we prepare the X and y dataframes for input into the model
- X represents the features and y is the output label

In [7]:
X = train.copy()
X.drop(['sales','date'], axis=1, inplace=True)
y = train['sales']

from sklearn.model_selection import train_test_split

# Split into train and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False) # Should not shuffle because time-series

print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_val: {y_val.shape}")

X_train: (730400, 6)
y_train: (730400,)
X_val: (182600, 6)
y_val: (182600,)


In [8]:
from xgboost import XGBRegressor

In [9]:
xgb = XGBRegressor(random_state=42)

In [10]:
def SMAPE(forecast, actual):
    """
    Symmetric Mean Absolute Percentage Error: from https://www.kaggle.com/enolac5/time-series-arima-dnn-xgboost-comparison
    """
    masked_arr = ~((forecast==0)&(actual==0))
    diff = abs(forecast[masked_arr] - actual[masked_arr])
    avg = (abs(forecast[masked_arr]) + abs(actual[masked_arr]))/2
    
    print('SMAPE Error Score: ' + str(round(sum(diff/avg)/len(forecast) * 100, 2)) + ' %')

In [11]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [12]:
y_hat = xgb.predict(X_val)
SMAPE(y_hat, y_val)

SMAPE Error Score: 46.3 %


Fit to all training data

In [13]:
xgb.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=2, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
print(f"X: {X.shape}")
print(f"y: {y.shape}")

X: (913000, 6)
y: (913000,)


In [15]:
X_test = test.copy()
X_test.drop(['id', 'date'], axis=1, inplace=True)

test_preds = xgb.predict(X_test)

In [16]:
sample_sub = sample_sub_original.copy()
sample_sub['sales'] = test_preds
sample_sub['sales'] = np.round(sample_sub['sales']).astype(int)
sample_sub.head()

,id,sales
0,0,13
1,1,15
2,2,15
3,3,17
4,4,18


In [17]:
sample_sub.to_csv('./submission.csv', index=False) # Public Score of 15.58

### Additional Feature Engineering
We can do some simple Feature Engineering to add more features for our model.
- We can one hot encode the months as we have noticed yearly seasonalities in our prior EDA in the previous notebooks.
- We can add a column `is_weekend` to model weekend trends
- We can add a column `avg_sales` to model the average sales for that particular item and store combination

We do this on a copy `train2` dataframe preserve our original `train` dataframe.

In [18]:
train2 = train.copy()
train2.head()

,date,store,item,sales,weekday,dayofyear,year,month
0,2013-01-01,1,1,13,1,1,2013,1
1,2013-01-02,1,1,11,2,2,2013,1
2,2013-01-03,1,1,14,3,3,2013,1
3,2013-01-04,1,1,13,4,4,2013,1
4,2013-01-05,1,1,10,5,5,2013,1


In [19]:
# One Hot Encode Months + drop first column to remove dummy variable trap
temp = pd.get_dummies(train2['month'], prefix='is_month', drop_first=True)

for col in temp.columns:
    train2[col] = temp[col]
    
train2.head()

,date,store,item,sales,weekday,dayofyear,year,month,is_month_2,is_month_3,is_month_4,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,is_month_11,is_month_12
0,2013-01-01,1,1,13,1,1,2013,1,0,0,0,0,0,0,0,0,0,0,0
1,2013-01-02,1,1,11,2,2,2013,1,0,0,0,0,0,0,0,0,0,0,0
2,2013-01-03,1,1,14,3,3,2013,1,0,0,0,0,0,0,0,0,0,0,0
3,2013-01-04,1,1,13,4,4,2013,1,0,0,0,0,0,0,0,0,0,0,0
4,2013-01-05,1,1,10,5,5,2013,1,0,0,0,0,0,0,0,0,0,0,0


Looks good so far. Now we can proceed to add the other features.

In [20]:
# Determines if day is a weekend
# Assumption: Friday is NOT a weekend
train2['is_weekend'] = train2['weekday'] // 5 # 0 for Mon-Fri (0-4), 1 for Sat (5) and Sun (6)
train2.head()

,date,store,item,sales,weekday,dayofyear,year,month,is_month_2,is_month_3,is_month_4,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,is_month_11,is_month_12,is_weekend
0,2013-01-01,1,1,13,1,1,2013,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2013-01-02,1,1,11,2,2,2013,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2013-01-03,1,1,14,3,3,2013,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2013-01-04,1,1,13,4,4,2013,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2013-01-05,1,1,10,5,5,2013,1,0,0,0,0,0,0,0,0,0,0,0,1


In [21]:
train2.tail()

,date,store,item,sales,weekday,dayofyear,year,month,is_month_2,is_month_3,is_month_4,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,is_month_11,is_month_12,is_weekend
912995,2017-12-27,10,50,63,2,361,2017,12,0,0,0,0,0,0,0,0,0,0,1,0
912996,2017-12-28,10,50,59,3,362,2017,12,0,0,0,0,0,0,0,0,0,0,1,0
912997,2017-12-29,10,50,74,4,363,2017,12,0,0,0,0,0,0,0,0,0,0,1,0
912998,2017-12-30,10,50,62,5,364,2017,12,0,0,0,0,0,0,0,0,0,0,1,1
912999,2017-12-31,10,50,82,6,365,2017,12,0,0,0,0,0,0,0,0,0,0,1,1


In [22]:
# Add in average sales for that particular store and item combination
from itertools import product

avg_sales = np.zeros(shape=(10, 50)) # (num_store, num_item)

for store, item in list(product(range(1, 11), range(1, 51))):
    avg_sales[store-1, item-1] = train2.query(f'store == {store} & item == {item}')['sales'].mean()

In [23]:
avg_sales_col = []

for _, row in train2.iterrows():
    store, item = row['store'], row['item']
    avg_sales_col.append(avg_sales[store-1, item-1])

In [24]:
train2['avg_sales'] = avg_sales_col
train2.tail()

,date,store,item,sales,weekday,dayofyear,year,month,is_month_2,is_month_3,...,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,is_month_11,is_month_12,is_weekend,avg_sales
912995,2017-12-27,10,50,63,2,361,2017,12,0,0,...,0,0,0,0,0,0,0,1,0,74.03724
912996,2017-12-28,10,50,59,3,362,2017,12,0,0,...,0,0,0,0,0,0,0,1,0,74.03724
912997,2017-12-29,10,50,74,4,363,2017,12,0,0,...,0,0,0,0,0,0,0,1,0,74.03724
912998,2017-12-30,10,50,62,5,364,2017,12,0,0,...,0,0,0,0,0,0,0,1,1,74.03724
912999,2017-12-31,10,50,82,6,365,2017,12,0,0,...,0,0,0,0,0,0,0,1,1,74.03724


Now we do the same for the `test2` dataframe

In [25]:
test2 = test.copy()

# One Hot Encode Months + drop first column to remove dummy variable trap
temp = pd.get_dummies(test2['month'], prefix='is_month', drop_first=True)

for col in temp.columns:
    test2[col] = temp[col]
    
# Determines if day is a weekend
# Assumption: Friday is NOT a weekend
test2['is_weekend'] = test2['weekday'] // 5 # 0 for Mon-Fri (0-4), 1 for Sat (5) and Sun (6)

# Recalculate avg_sales column for test dataframe
avg_sales_col = []

for _, row in test2.iterrows():
    store, item = row['store'], row['item']
    avg_sales_col.append(avg_sales[store-1, item-1])
test2['avg_sales'] = avg_sales_col
test2.tail()

,date,id,store,item,weekday,dayofyear,year,month,is_month_2,is_month_3,is_weekend,avg_sales
44995,2018-03-27,44995,10,50,1,86,2018,3,0,1,0,74.03724
44996,2018-03-28,44996,10,50,2,87,2018,3,0,1,0,74.03724
44997,2018-03-29,44997,10,50,3,88,2018,3,0,1,0,74.03724
44998,2018-03-30,44998,10,50,4,89,2018,3,0,1,0,74.03724
44999,2018-03-31,44999,10,50,5,90,2018,3,0,1,1,74.03724


However, we note that the test dataframe does not have columns `is_month_4` to `is_month_12`, as the test data spans only January to March. We can manually add in 0 for these columns.

In [26]:
test2 = test.copy()

# One Hot Encode Months + drop first column to remove dummy variable trap
temp = pd.get_dummies(test2['month'], prefix='is_month', drop_first=True)

for col in temp.columns:
    test2[col] = temp[col]

# add in columns manually (test data spans Jan-Mar only)
for i in range(4, 13):
    test2[f'is_month_{i}'] = 0
    
# Determines if day is a weekend
# Assumption: Friday is NOT a weekend
test2['is_weekend'] = test2['weekday'] // 5 # 0 for Mon-Fri (0-4), 1 for Sat (5) and Sun (6)

# Recalculate avg_sales column for test dataframe
avg_sales_col = []

for _, row in test2.iterrows():
    store, item = row['store'], row['item']
    avg_sales_col.append(avg_sales[store-1, item-1])
test2['avg_sales'] = avg_sales_col
test2.tail()

,date,id,store,item,weekday,dayofyear,year,month,is_month_2,is_month_3,...,is_month_5,is_month_6,is_month_7,is_month_8,is_month_9,is_month_10,is_month_11,is_month_12,is_weekend,avg_sales
44995,2018-03-27,44995,10,50,1,86,2018,3,0,1,...,0,0,0,0,0,0,0,0,0,74.03724
44996,2018-03-28,44996,10,50,2,87,2018,3,0,1,...,0,0,0,0,0,0,0,0,0,74.03724
44997,2018-03-29,44997,10,50,3,88,2018,3,0,1,...,0,0,0,0,0,0,0,0,0,74.03724
44998,2018-03-30,44998,10,50,4,89,2018,3,0,1,...,0,0,0,0,0,0,0,0,0,74.03724
44999,2018-03-31,44999,10,50,5,90,2018,3,0,1,...,0,0,0,0,0,0,0,0,1,74.03724


Fit the model on all the data (X and y)

In [27]:
X = train2.copy()
X.drop(['sales','date'], axis=1, inplace=True)
y = train2['sales']

xgb.fit(X, y)
print(f"X: {X.shape}")
print(f"y: {y.shape}")

X: (913000, 19)
y: (913000,)


In [28]:
X_test = test2.copy()
X_test.drop(['id', 'date'], axis=1, inplace=True)

test_preds = xgb.predict(X_test)

In [29]:
sample_sub = sample_sub_original.copy()
sample_sub['sales'] = test_preds
sample_sub['sales'] = np.round(sample_sub['sales']).astype(int)
sample_sub.head()

,id,sales
0,0,11
1,1,14
2,2,14
3,3,15
4,4,17


In [30]:
sample_sub.to_csv('./submission.csv', index=False) # Public Score of 14.15!!

### Hyperparameter Tuning
We use Bayesian Search Cross Validation provided by the scikit-optimize package

The parameters are optimized via cross-validation search, using a parameter grid.

[BayesSearchCV](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html) tries out a fixed number of parameter settings sampled from specified distributions. Unlike GridSearchCV, it does not try out all possible parameter settings from the parameter grid.

In [31]:
from skopt import BayesSearchCV

### From the XGBoost [documentation](https://xgboost.readthedocs.io/en/latest/parameter.html):

1. max_depth [default=6]
    - Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. 0 is only accepted in lossguided growing policy when tree_method is set as hist or gpu_hist and it indicates no limit on depth. Beware that XGBoost aggressively consumes memory when training a deep tree.
    - range: [0,∞] (0 is only accepted in lossguided growing policy when tree_method is set as hist or gpu_hist)


2. gamma [default=0, alias: min_split_loss]
    - Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
    - range: [0,∞]


3. eta [default=0.3, alias: learning_rate]
    - Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
    - range: [0,1]
   
   
4. subsample [default=1]

    - Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.

    - range: (0,1]
    
5. min_child_weight [default=1]

    - Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning. In linear regression task, this simply corresponds to minimum number of instances needed to be in each node. The larger min_child_weight is, the more conservative the algorithm will be.

    - range: [0,∞]
    
6. lambda [default=1, alias: reg_lambda]

    - L2 regularization term on weights. Increasing this value will make model more conservative.
    
7. tree_method [default= auto]

    - The tree construction algorithm used in XGBoost.
    
    - Setting to `gpu_hist` enables training to be accelerated via CUDA-capable GPUs.

In [32]:
param_grid = {
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'gamma': np.arange(0, 50, 0.001),
    'eta': np.arange(0.001, 0.3 ,0.001), # learning rate
    'subsample': np.arange(0.1, 0.5, 0.01),
    'min_child_weight': np.arange(0.01, 0.5, 0.01),
    'lambda': np.arange(0, 10, 0.1),
    'tree_method': ['gpu_hist']
}

In [33]:
xgb_opt = BayesSearchCV(XGBRegressor(random_state=42), param_grid, n_iter=500, cv=5, random_state=42, verbose=1)

After hyperparameter tuning is completed, we can view the best parameters found.
We also fit the new model on the entire dataset again to obtain our new `submission.csv`
- We can reuse `X` and `y` from the previous model from the `Additional Features` section.

In [34]:
xgb_opt.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   45.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.5s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.9s finished


BayesSearchCV(cv=5,
              estimator=XGBRegressor(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49]),
                             'subsample': array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 

In [35]:
xgb_opt.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.08, gamma=21.255,
             gpu_id=0, importance_type='gain', interaction_constraints='',
             lambda=8.0, learning_rate=0.0799999982, max_delta_step=0,
             max_depth=7, min_child_weight=0.24000000000000002, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=2,
             num_parallel_tree=1, random_state=42, reg_alpha=0, reg_lambda=8,
             scale_pos_weight=1, subsample=0.4099999999999998,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

By default, `xgb_opt` will refit the model with the best parameters: `refit=True`; we can directly call predict on `xgb_opt`.

In [36]:
test_preds = xgb_opt.predict(X_test)

In [37]:
sample_sub = sample_sub_original.copy()
sample_sub['sales'] = test_preds
sample_sub['sales'] = np.round(sample_sub['sales']).astype(int)
sample_sub.head()

,id,sales
0,0,12
1,1,14
2,2,14
3,3,15
4,4,16


In [38]:
sample_sub.to_csv('./submission.csv', index=False) # Public Score of around 14.08!!